In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import os

In [46]:
stock_file = '../data/stock_data_with_metrics.csv'  # Adjust path if needed
if os.path.exists(stock_file):
    stock_df = pd.read_csv(stock_file)
    stock_df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
else:
    raise FileNotFoundError(f"{stock_file} not found. Ensure Task 2 output is available.")

# Load news data (assumed file structure)
news_file = '../data/raw_analyst_ratings.csv'  # Adjust path as per your directory structure
if os.path.exists(news_file):
    news_df = pd.read_csv(news_file)
    news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce').dt.date
    news_df['date'] = pd.to_datetime(news_df['date'])
    print("Initial news_df shape:", news_df.shape)
    print("Initial news_df head:\n", news_df.head())
    
else:
    raise FileNotFoundError(f"{news_file} not found. Please provide the news dataset.")

Initial news_df shape: (1407328, 6)
Initial news_df head:
    Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

        date stock  
0 2020-06-05     A  
1 2020-06-03     A  
2 2020-05-26     A

In [47]:
# Step 2: Normalize dates and set indices
# Set multi-index for stock_df using 'Date' and 'Stock'
stock_df.set_index(['Date', 'Stock'], inplace=True)
# Drop rows with NaN in 'Daily_Return' to ensure meaningful analysis
stock_df = stock_df.dropna(subset=['Daily_Return'])
print("Stock_df shape after filtering NaN:", stock_df.shape)
print("Stock_df index levels:", stock_df.index.names)
print("Valid dates (first 5):", stock_df.index.get_level_values('Date').unique()[:5])
print("Valid stocks:", stock_df.index.get_level_values('Stock').unique())

# Filter news_df to include only dates present in stock_df
valid_dates = stock_df.index.get_level_values('Date').date  # Extract date part for comparison
valid_dates = pd.to_datetime(valid_dates)  # Convert back to datetime
valid_stocks = stock_df.index.get_level_values('Stock').unique()
# Temporarily disable stock filter due to mismatch
news_df = news_df[news_df['date'].isin(valid_dates)]
print("News_df shape after filtering:", news_df.shape)
print("News_df head after filtering:\n", news_df.head())

# Check if news_df is empty after filtering
if news_df.empty:
    raise ValueError("No matching dates found between news_df and stock_df. Please check the date ranges.")


Stock_df shape after filtering NaN: (45421, 17)
Stock_df index levels: ['Date', 'Stock']
Valid dates (first 5): DatetimeIndex(['1980-12-15', '1980-12-16', '1980-12-17', '1980-12-18',
               '1980-12-19'],
              dtype='datetime64[ns]', name='Date', freq=None)
Valid stocks: Index(['AAPL', 'MSFT', 'AMZN', 'NVDA', 'GOOG', 'TSLA', 'META'], dtype='object', name='Stock')
News_df shape after filtering: (55230, 6)
News_df head after filtering:
    Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzin

In [48]:
# Step 3: Perform sentiment analysis on news headlines
def get_sentiment(headline):
    blob = TextBlob(str(headline))
    sentiment = blob.sentiment.polarity
    return sentiment

news_df['Sentiment_Score'] = news_df['headline'].apply(get_sentiment)
print("News_df with Sentiment_Score head:\n", news_df.head())

# Aggregate sentiment scores by date and stock (average if multiple articles)
daily_sentiment = news_df.groupby(['date', 'stock'])['Sentiment_Score'].mean().reset_index()
daily_sentiment.set_index(['date', 'stock'], inplace=True)
print("Daily sentiment shape:", daily_sentiment.shape)
print("Daily sentiment head:\n", daily_sentiment.head())

# Save intermediate results
daily_sentiment.to_csv('../data/daily_sentiment_scores.csv')
print("Daily sentiment scores saved to '../data/daily_sentiment_scores.csv'")

News_df with Sentiment_Score head:
    Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

        date stock  Sentiment_Score  
0 2020-06-05     A              0.0  
1 2020-06-03     A          

In [53]:
# Step 4: Merge with stock data for correlation analysis
merged_df = stock_df.reset_index().merge(daily_sentiment.reset_index(), 
                                        left_on=['Date', 'Stock'], 
                                        right_on=['date', 'stock'], 
                                        how='inner')
merged_df.set_index(['Date', 'Stock'], inplace=True)
print("Merged_df shape:", merged_df.shape)
print("Merged_df head:\n", merged_df)
# print(merged_df.info())

Merged_df shape: (14, 20)
Merged_df head:
                         Open        High         Low       Close   Adj Close  \
Date       Stock                                                               
2020-06-02 NVDA     8.796250    8.837250    8.658000    8.825250    8.795022   
2020-06-04 GOOG    71.519997   71.947998   70.236504   70.609001   70.528740   
2020-06-05 GOOG    70.658501   72.252502   70.300003   71.919502   71.837753   
2020-06-08 GOOG    71.116997   72.399498   71.116997   72.330498   72.248283   
           NVDA     8.877250    8.889750    8.685750    8.805000    8.778847   
2020-06-09 AMZN   126.472000  131.321503  126.250000  130.042999  130.042999   
           GOOG    72.267998   73.400002   72.160500   72.807999   72.725243   
           AAPL    83.035004   86.402496   83.002502   85.997498   83.889359   
           NVDA     8.800000    9.109750    8.751250    9.046000    9.019129   
2020-06-10 GOOG    72.976997   73.712952   72.813499   73.292503   73.209198 

In [56]:
# Step 5: Correlation analysis per stock
correlation_results = {}
print("Available stocks in merged_df:", merged_df.index.get_level_values('Stock').unique())
print("Merged_df head:\n", merged_df.head())
for stock in ['NVDA', 'GOOG', 'AMZN', 'AAPL', 'TSLA']:
    stock_data = merged_df.xs(stock, level='Stock')
    if not stock_data.empty:
        correlation = stock_data['Daily_Return'].corr(stock_data['Sentiment_Score'], method='pearson')
        correlation_results[stock] = correlation
    else:
        print(f"No data found for stock {stock} in merged_df")

# Print correlation results
print("\nPearson Correlation Coefficients by Stock:")
for stock, corr in correlation_results.items():
    # Interpret correlation strength
    if abs(corr) >= 0.7:
        strength = "Strong"
    elif abs(corr) >= 0.5:
        strength = "Moderate"
    elif abs(corr) >= 0.3:
        strength = "Weak"
    else:
        strength = "Negligible"
    print(f"{stock}: Correlation = {corr:.4f}, Strength = {strength}")



Available stocks in merged_df: Index(['NVDA', 'GOOG', 'AMZN', 'AAPL', 'TSLA'], dtype='object', name='Stock')
Merged_df head:
                        Open       High        Low      Close  Adj Close  \
Date       Stock                                                          
2020-06-02 NVDA    8.796250   8.837250   8.658000   8.825250   8.795022   
2020-06-04 GOOG   71.519997  71.947998  70.236504  70.609001  70.528740   
2020-06-05 GOOG   70.658501  72.252502  70.300003  71.919502  71.837753   
2020-06-08 GOOG   71.116997  72.399498  71.116997  72.330498  72.248283   
           NVDA    8.877250   8.889750   8.685750   8.805000   8.778847   

                       Volume  Dividends  Stock Splits     SMA_50        RSI  \
Date       Stock                                                               
2020-06-02 NVDA   391320000.0        0.0           0.0   7.407945  63.175825   
2020-06-04 GOOG    29686000.0        0.0           0.0  64.943405  58.926016   
2020-06-05 GOOG    34698000.

c:\Users\lenovo\Desktop\AIM\10_academy\Week_1_challenge\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:3037: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\lenovo\Desktop\AIM\10_academy\Week_1_challenge\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2894: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\lenovo\Desktop\AIM\10_academy\Week_1_challenge\venv\Lib\site-packages\numpy\lib\_function_base_impl.py:2894: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [57]:
# Save the merged dataset with correlation results
merged_df.to_csv('../data/sentiment_stock_correlation.csv')
print("Merged dataset with correlation results saved to '../data/sentiment_stock_correlation.csv'")

Merged dataset with correlation results saved to '../data/sentiment_stock_correlation.csv'
